In [1]:
# getting the requisite training data files from where we left in Part II

%load_ext autoreload
%autoreload 2

In [2]:
import noshow_lib.util as utils
utils.file_config
# I have changed the configuration file in the util.py file

{'raw_data_path': 'C:/Vidit/PhD/Fall 2018/CMSC 643 - Hector/Project 2/cmsc643_noshow-master/part2_data/data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'processed_data_path': 'C:/Vidit/PhD/Fall 2018/CMSC 643 - Hector/Project 2/cmsc643_noshow-master/part2_data/processed_dir',
 'train_csv': 'train_set.csv',
 'test_csv': 'test_set.csv',
 'objstore_path': 'C:/Vidit/PhD/Fall 2018/CMSC 643 - Hector/Project 2/cmsc643_noshow-master/part2_data/objects',
 'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl'}

In [3]:
file_config = utils.file_config

In [4]:
import noshow_lib.preprocess as preprocess
train_X, train_y = preprocess.load_train_data(config=file_config)

In [5]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


# Part III: Ensembles and Final Result

## AdaBoost

Train an AdaBoost classifier and compare its performance to results obtained in Part II using 10 fold CV.

In [7]:
# AdaBoost code goes here
from sklearn.ensemble import AdaBoostClassifier
num_trees = 30
seed = 7
model = AdaBoostClassifier(n_estimators = num_trees, random_state = seed)
model.fit(train_X, train_y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=30, random_state=7)

In [11]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, train_X, train_y, scoring='accuracy', cv=10)
print("AdaBoost Accuracy metric:", scores.mean())

AdaBoost Accuracy metric: 0.7976824222007982


In [12]:
auc_scores = cross_val_score(model, train_X, train_y, scoring='roc_auc', cv=10)
print("AdaBoost AUC metric:", auc_scores.mean())

AdaBoost AUC metric: 0.7255392582784536


## xgBoost

Train an xgBoost classifier and compare its performance to results in Part II using 10 fold CV. `sklearn` has a gradient boosting model included http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html which you can use. The `xgboost` package https://xgboost.readthedocs.io/en/latest/python/python_intro.htmlhas a wrapper you can use with sklearn as well https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn. The latter is more efficient at training time.

In [13]:
# xgboost code here

from xgboost import XGBClassifier
model_xg = XGBClassifier()
model_xg.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
scores_xg = cross_val_score(model_xg, train_X, train_y, scoring='accuracy', cv=10)
print("xgBoost Accuracy metric:", scores_xg.mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

KeyboardInterrupt: 

In [ ]:
auc_scores_xg = cross_val_score(model, train_X, train_y, scoring='roc_auc', cv=10)
print("xgBoost AUC metric:", auc_scores_xg.mean())
# Not executing this!

## Stacking

Choose a set of 5 or so classifiers. Write a function that trains an ensemble using stacking

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import pandas as pd

def build_stack_ensemble(X, y):
    
    # create train/validation sets
    # using StratifiedShuffleSplit
    
    split = StratifiedShuffleSplit(n_splits=1, 
                                   test_size=20000,
                                   random_state = 1234)
    
    for train_index, validation_index in split.split(X, y):
        train_X, val_X = X.iloc[train_index], X.iloc[validation_index]
        train_y, val_y = y.iloc[train_index], y.iloc[validation_index]
   
    
    # train classifiers in ensemble using train set
    
    dt = DecisionTreeClassifier()                      # Decision Tree
    dt.fit(train_X, train_y)
    
    rf = RandomForestClassifier()                      # Random Forest
    rf.fit(train_X, train_y)
    
    lsvm = LinearSVC(dual=False)                       # Linear SVM
    lsvm.fit(train_X, train_y)
    
    svk = SVC(kernel='rbf', gamma='auto')              # SVM with Radial Basis Kernel classifier
    svk.fit(train_X, train_y)
    
    # create new feature matrix for validation
    # set by getting predictions from the ensemble
    # classifiers
    
    # Getting Predictions
    dt_val = dt.predict(val_X)
    rf_val = rf.predict(val_X)
    lsvm_val = lsvm.predict(val_X)
    svk_val = svk.predict(val_X)
    
    dt_v = pd.DataFrame(dt_val)
    rf_v = pd.DataFrame(rf_val)
    lsvm_v = pd.DataFrame(lsvm_val)
    svk_v = pd.DataFrame(svk_val)
    
    df_val = pd.concat([dt_v, rf_v, lsvm_v, svk_v],axis=1)              #  new feature matrix
    
    # train logistic regression classifier on
    # new feature matrix
    
    combined_model = LogisticRegression()
    combined_model.fit(df_val, val_y)
    
    # return all trained classifiers
    return dt, rf, lsvm, svk, combined_model

Use 10-fold cross validation to measure performance of your stacked classifier. See Part II solution to see how to roll your own sklearn classifier along with http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator

## Final Result

Choose a single model based on all previous project steps. Train this model on the complete training dataset and measure it's performance on the held out test set.

Compare to the 10-fold CV estimate you got previously.

In [3]:
# final result goes here

import noshow_lib.util as utils
import noshow_lib.preprocess as preprocess
train_X, train_y = preprocess.load_train_data(config=file_config)

dt, rf, lsvm, svk, combined_model = build_stack_ensemble(train_X,train_y)
# Choosing the combined model
cm = combined_model.fit(train_X, train_y)   # Training
test_X, test_y = preprocess.load_test_data(config=file_config)  # Fetching the test set
performance = cm.score(test_X, test_y)
print("Mean accuracy on the held out test set", performance)